코드 전체 구조
1. 필요한 라이브러리 import
2. 데이터 로드 및 전처리 함수들
3. 토픽 모델링 함수들
4. 평가 지표 계산 함수들
5. LLM 평가 관련 함수들
6. 상관관계 분석 함수
7. summarize_results 함수
8. 메인 실행 코드

In [1]:
# 필요한 모듈 import
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gensim
from gensim import models, corpora, matutils
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
import openai
import os
import re
import json
import time
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from scipy.stats import pearsonr, spearmanr, f_oneway, kruskal
from tenacity import retry, stop_after_attempt, wait_random_exponential
from bertopic import BERTopic
from transformers import BertTokenizer, BertModel 
from itertools import combinations
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
from scipy.spatial.distance import cityblock, jensenshannon
from transformers import BertTokenizer, BertModel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\user\anaconda3\envs\topic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 데이터셋 선정 함수
def load_data(file_path, sample_size=80):
    df = pd.read_csv(file_path, header=None, names=['text'])
    texts = df['text'].astype(str)

    # 샘플링
    if len(texts) > sample_size:
        texts = texts.sample(n=sample_size, random_state=42)

    print(f"Loaded {len(texts)} texts from {file_path}")
    return texts.tolist()

# 데이터셋 로드
datasets = {
    'academy': {
        'business': load_data('data/academy/business.csv'),
        'ACL': load_data('data/academy/ACL.csv'),
        'covid': load_data('data/academy/covid.csv')
    },
    'media': {
        'clothing_review': load_data('data/media/clothing_review.csv'),
        'vaccine_tweets': load_data('data/media/vaccine_tweets.csv'),
        'reddit_comments': load_data('data/media/reddit_comments.csv')
    },
    'news': {
        'newsgroups': load_data('data/news/20newsgroups.csv'),
        'agnews': load_data('data/news/agnews.csv'),
        'Huffpost': load_data('data/news/Huffpost.csv')
    }
}

# VAE 모델 정의
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)

        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

def perform_topic_modeling(data, num_topics, model_type):
    # Ensure all data is of type string
    data = [str(doc) for doc in data if isinstance(doc, str) or pd.notna(doc)]

    # Check if num_topics is greater than the number of documents
    if num_topics > len(data):
        print(f"Adjusting num_topics from {num_topics} to {len(data)}")
        num_topics = len(data)  # Adjust to the number of documents

    # Common vectorizer
    vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')
    doc_term_matrix = vectorizer.fit_transform(data)

    # GPU usage setting
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if model_type == 'LDA':
        # Create corpus and dictionary for LDA
        corpus = matutils.Sparse2Corpus(doc_term_matrix, documents_columns=False)
        id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

        # Create LDA model
        lda_model = models.LdaMulticore(
            corpus=corpus,
            id2word=id2word,
            num_topics=num_topics,
            workers=2,  # Number of cores to use
            passes=10,
            random_state=42
        )

        return lda_model, vectorizer

    elif model_type == 'BERTopic':
        # Create BERTopic model
        bertopic_model = BERTopic(nr_topics=num_topics)
        
        # Ensure documents is a Series
        if not isinstance(data, pd.Series):
            data = pd.Series(data)
        
        bertopic_topics, _ = bertopic_model.fit_transform(data)

        return bertopic_model, None  # BERTopic uses its own vectorizer

    elif model_type == 'VAE':
        # VAE part
        input_dim = doc_term_matrix.shape[1]
        hidden_dim = 256
        latent_dim = num_topics

        vae_model = VAE(input_dim, hidden_dim, latent_dim).to(device)
        optimizer = torch.optim.Adam(vae_model.parameters(), lr=1e-3)

        # VAE training
        num_epochs = 2  # 10
        batch_size = 128

        for epoch in range(num_epochs):
            for i in range(0, doc_term_matrix.shape[0], batch_size):
                batch = torch.FloatTensor(doc_term_matrix[i:i+batch_size].toarray()).to(device)
                batch = batch / batch.max()  # Normalize batch to between 0 and 1

                optimizer.zero_grad()
                recon_batch, mu, logvar = vae_model(batch)
                loss = vae_loss(recon_batch, batch, mu, logvar)
                loss.backward()
                optimizer.step()

        # Extract topic words from latent vectors
        latent_vectors = []
        doc_term_matrix = vectorizer.transform(data)
        with torch.no_grad():
            for i in range(0, doc_term_matrix.shape[0], 128):
                batch = torch.FloatTensor(doc_term_matrix[i:i+128].toarray()).to(device)
                mu, logvar = vae_model.encode(batch)
                z = vae_model.reparameterize(mu, logvar)
                latent_vectors.append(z.cpu().numpy())

        latent_vectors = np.vstack(latent_vectors)
        kmeans = KMeans(n_clusters=num_topics, random_state=42).fit(latent_vectors)

        topics = [[] for _ in range(num_topics)]
        for idx, label in enumerate(kmeans.labels_):
            doc = data[idx]
            topics[label].extend(doc.split())

        # Extract top 10 words for each topic
        topics = [list(pd.Series(words).value_counts().index[:10]) for words in topics]

        # Return VAE model, vectorizer, and topic words
        return vae_model, vectorizer, topics

Loaded 80 texts from data/academy/business.csv
Loaded 80 texts from data/academy/ACL.csv
Loaded 80 texts from data/academy/covid.csv
Loaded 80 texts from data/media/clothing_review.csv
Loaded 80 texts from data/media/vaccine_tweets.csv
Loaded 80 texts from data/media/reddit_comments.csv
Loaded 80 texts from data/news/20newsgroups.csv
Loaded 80 texts from data/news/agnews.csv
Loaded 80 texts from data/news/Huffpost.csv


In [4]:
# 유틸리티 함수들
def extract_topics(model_type, model, num_topics, topics=None):
    if model_type == 'LDA':
        return [model.show_topic(i, topn=10) for i in range(num_topics)]
    elif model_type == 'BERTopic':
        return [model.get_topic(i) for i in range(num_topics) if model.get_topic(i)]
    elif model_type == 'VAE':
        return topics  # VAE의 경우 이미 추출된 토픽을 사용
    else:
        raise ValueError("Invalid model type")

# 토픽 간 구별성 지표(distinct) 측정 함수
def compute_distinctiveness_score(phi, word_embeddings, reduced_dim=50):
    # Step 2: 차원 축소 (선택 사항)
    def reduce_dimensionality(embeddings, n_components):
        pca = PCA(n_components=n_components)
        reduced_embeddings = pca.fit_transform(embeddings)
        return reduced_embeddings

    # 차원 축소 적용
    word_embeddings_reduced = reduce_dimensionality(word_embeddings, reduced_dim)

    # Step 3: 토픽 임베딩 계산
    def compute_topic_embeddings(phi, word_embeddings):
        num_topics = phi.shape[0]
        topic_embeddings = np.zeros((num_topics, word_embeddings.shape[1]))
        for i in range(num_topics):
            weights = phi[i]
            weights = weights / weights.sum()  # 가중치 정규화
            topic_embedding = np.dot(weights, word_embeddings)
            topic_embeddings[i] = topic_embedding
        return topic_embeddings

    topic_embeddings = compute_topic_embeddings(phi, word_embeddings_reduced)

    # Step 4: 토픽 간 거리 계산
    def compute_distance_matrix(topic_embeddings, metric='cosine'):
        if metric == 'cosine':
            distance_matrix = cosine_distances(topic_embeddings)
        elif metric == 'euclidean':
            distance_matrix = euclidean_distances(topic_embeddings)
        elif metric == 'manhattan':
            num_topics = topic_embeddings.shape[0]
            distance_matrix = np.zeros((num_topics, num_topics))
            for i in range(num_topics):
                for j in range(i+1, num_topics):
                    dist = cityblock(topic_embeddings[i], topic_embeddings[j])
                    distance_matrix[i, j] = dist
                    distance_matrix[j, i] = dist
        else:
            raise ValueError("지원하지 않는 거리 척도입니다.")
        return distance_matrix

    # 거리 행렬 계산 (코사인 거리 사용)
    distance_matrix = compute_distance_matrix(topic_embeddings, metric='cosine')

    # Step 5: 구별성 점수 계산
    def compute_distinctiveness_score(distance_matrix):
        num_topics = distance_matrix.shape[0]
        # 상삼각 행렬의 값만 사용하여 평균 거리 계산
        triu_indices = np.triu_indices(num_topics, k=1)
        distances = distance_matrix[triu_indices]
        avg_distance = distances.mean()
        return avg_distance

    distinctiveness_score = compute_distinctiveness_score(distance_matrix)
    return distinctiveness_score

# Jensen-Shannon Divergence (JSD) 계산 함수
def compute_pairwise_jsd(phi):
    num_topics = phi.shape[0]
    jsd_matrix = np.zeros((num_topics, num_topics))
    for i in range(num_topics):
        for j in range(i+1, num_topics):
            jsd = jensenshannon(phi[i], phi[j])
            jsd_matrix[i, j] = jsd
            jsd_matrix[j, i] = jsd  # 대칭으로 만듭니다.
    return jsd_matrix

def compute_average_jsd(jsd_matrix):
    num_topics = jsd_matrix.shape[0]
    # 상삼각 행렬의 값만 사용하여 평균을 계산합니다.
    triu_indices = np.triu_indices(num_topics, k=1)
    avg_jsd = jsd_matrix[triu_indices].mean()
    return avg_jsd

# Hellinger Distance 계산 함수
def compute_pairwise_hellinger(phi):
    num_topics = phi.shape[0]
    hellinger_matrix = np.zeros((num_topics, num_topics))
    for i in range(num_topics):
        sqrt_phi_i = np.sqrt(phi[i])
        for j in range(i+1, num_topics):
            sqrt_phi_j = np.sqrt(phi[j])
            distance = np.linalg.norm(sqrt_phi_i - sqrt_phi_j) / np.sqrt(2)
            hellinger_matrix[i, j] = distance
            hellinger_matrix[j, i] = distance  # 대칭으로 만듭니다.
    return hellinger_matrix

def compute_average_hellinger(hellinger_matrix):
    num_topics = hellinger_matrix.shape[0]
    # 상삼각 행렬의 값만 사용하여 평균을 계산합니다.
    triu_indices = np.triu_indices(num_topics, k=1)
    avg_hellinger = hellinger_matrix[triu_indices].mean()
    return avg_hellinger

# 평가 지표 계산 통합 함수

# BERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)

def calculate_evaluation_metrics(model, data, model_type, vectorizer=None, num_topics=10, topics=None):
    if model_type == 'LDA':
        phi = model.get_topics()
    elif model_type == 'BERTopic':
        phi = np.array([model.get_topic(i) for i in range(num_topics)])
    elif model_type == 'VAE':
        phi = model.fc4.weight.data.cpu().numpy()
    else:
        raise ValueError("Invalid model type")

    # BERT를 사용하여 단어 임베딩 생성
    vocab = vectorizer.get_feature_names() if vectorizer else list(set([word for topic in topics for word in topic]))
    word_embeddings = []
    
    for word in vocab:
        inputs = tokenizer(word, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        word_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [CLS] 토큰의 임베딩 사용
        word_embeddings.append(word_embedding)

    word_embeddings = np.vstack(word_embeddings)

    # Distinctiveness 계산
    distinctiveness = compute_distinctiveness_score(phi, word_embeddings)

    # JSD 계산
    jsd_matrix = compute_pairwise_jsd(phi)
    avg_jsd = compute_average_jsd(jsd_matrix)

    # Hellinger Distance 계산
    hellinger_matrix = compute_pairwise_hellinger(phi)
    avg_hellinger = compute_average_hellinger(hellinger_matrix)

    return distinctiveness, avg_jsd, avg_hellinger

In [ ]:
def correlation_analysis(metrics_df):
    from scipy.stats import pearsonr, spearmanr

    # 지표 목록
    metric_names = ['Distinctiveness', 'JSD', 'Hellinger']

    # 모델별, 토픽 수별로 상관관계 계산
    for model in metrics_df['Model'].unique():
        for num_topics in metrics_df['Num_Topics'].unique():
            subset = metrics_df[(metrics_df['Model'] == model) & (metrics_df['Num_Topics'] == num_topics)]
            if len(subset) < 2:
                continue  # 상관계수를 계산하기 위한 데이터가 충분하지 않으면 건너뜀
            print(f"\n상관관계 분석 - 모델: {model}, 토픽 수: {num_topics}")
            for i in range(len(metric_names)):
                for j in range(i+1, len(metric_names)):
                    metric1 = metric_names[i]
                    metric2 = metric_names[j]
                    pearson_corr, p_value_pearson = pearsonr(subset[metric1], subset[metric2])
                    spearman_corr, p_value_spearman = spearmanr(subset[metric1], subset[metric2])
                    print(f"{metric1} vs {metric2}:")
                    print(f"  Pearson: 상관계수 = {pearson_corr:.4f}, p-value = {p_value_pearson:.4f}")
                    print(f"  Spearman: 상관계수 = {spearman_corr:.4f}, p-value = {p_value_spearman:.4f}")

    # 전체 데이터에 대한 상관관계 분석
    print("\n전체 데이터에 대한 상관관계 분석:")
    for i in range(len(metric_names)):
        for j in range(i+1, len(metric_names)):
            metric1 = metric_names[i]
            metric2 = metric_names[j]
            pearson_corr, p_value_pearson = pearsonr(metrics_df[metric1], metrics_df[metric2])
            spearman_corr, p_value_spearman = spearmanr(metrics_df[metric1], metrics_df[metric2])
            print(f"{metric1} vs {metric2}:")
            print(f"  Pearson: 상관계수 = {pearson_corr:.4f}, p-value = {p_value_pearson:.4f}")
            print(f"  Spearman: 상관계수 = {spearman_corr:.4f}, p-value = {p_value_spearman:.4f}")

# LLM 평가 함수
def call_openai_api(prompt: str, max_tokens: int = 3000) -> str:
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    full_response = ""
    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are an expert in topic modeling and text analysis. Your task is to evaluate the coherence of topics based on provided documents."
                    },
                    {
                        "role": "user",
                        "content": prompt + ("\n\nContinue from: " + full_response if full_response else "")
                    }
                ],
                temperature=0,
                max_tokens=max_tokens,
                top_p=1,
                frequency_penalty=0.1,
                presence_penalty=0.1,
            )
            chunk = response.choices[0].message.content
            full_response += chunk
            
            if not response.choices[0].finish_reason == "length":
                break
            
            prompt = "Continue the previous response:"
        except openai.error.RateLimitError:
            print("Rate limit exceeded. Retrying...")
            time.sleep(60)  # Wait for 60 seconds before retrying
        except openai.error.AuthenticationError:
            print("Authentication error. Check your API key.")
            raise
        except Exception as e:
            print(f"Unexpected error: {e}")
            raise
    return full_response

def llm_evaluation(topics, documents, model="gpt-4o-mini"):
    scores = []
    feedbacks = []

    if not isinstance(documents, list):
        documents = list(documents)

    for topic_words in topics:
        topic = ', '.join(topic_words)
        docs_sample = documents[:3]
        prompt = f"""
주어진 토픽과 관련 문서를 평가해주세요. 다음 기준에 따라 1-10 척도로 점수를 매겨주세요:
일관성: 토픽 내 단어들이 의미적으로 얼마나 연관되어 있는가?

토픽: {topic}
관련 문서 샘플:
{docs_sample}

일관성에 대해 1-10 점수를 매기고, 간단한 설명을 덧붙여주세요.
"""
        try:
            evaluation = call_openai_api(prompt)

            match = re.search(r'(\d+)', evaluation)
            if match:
                topic_score = int(match.group(1))
                scores.append(topic_score)
                feedbacks.append(evaluation)
            else:
                print(f"점수를 추출할 수 없습니다: {evaluation}")
        except Exception as e:
            print(f"Unexpected error: {e}")
            raise

    return scores, feedbacks

def analyze_llm_results(llm_df):
    llm_df['LLM_Avg_Score'] = llm_df['LLM_Scores'].apply(lambda scores: np.mean([s for s in scores if s is not None]))
    llm_df['LLM_Std_Score'] = llm_df['LLM_Scores'].apply(lambda scores: np.std([s for s in scores if s is not None]))
    llm_df['LLM_Median_Score'] = llm_df['LLM_Scores'].apply(lambda scores: np.median([s for s in scores if s is not None]))

    print("\nLLM 평가 결과:")
    print(llm_df[['Domain', 'Model', 'Num_Topics', 'LLM_Avg_Score', 'LLM_Std_Score', 'LLM_Median_Score']])

def llm_auto_metric_correlation(metrics_df, llm_df):
    merged_df = pd.merge(metrics_df, llm_df, on=['Domain', 'Dataset', 'Model', 'Num_Topics'])

    metric_names = ['Distinctiveness', 'JSD', 'Hellinger']
    for metric in metric_names:
        valid_idx = merged_df['LLM_Avg_Score'].notnull()
        pearson_corr, p_value_pearson = pearsonr(merged_df.loc[valid_idx, metric], merged_df.loc[valid_idx, 'LLM_Avg_Score'])
        spearman_corr, p_value_spearman = spearmanr(merged_df.loc[valid_idx, metric], merged_df.loc[valid_idx, 'LLM_Avg_Score'])
        print(f"\nLLM 평가 점수와 {metric}의 상관관계:")
        print(f"Pearson: 상관계수 = {pearson_corr:.4f}, p-value = {p_value_pearson:.4f}")
        print(f"Spearman: 상관계수 = {spearman_corr:.4f}, p-value = {p_value_spearman:.4f}")

def verify_llm_consistency(topics, documents, n_repeats=5):
    all_scores = []
    for _ in range(n_repeats):
        scores, _ = llm_evaluation(topics, documents)
        all_scores.append(scores)
    all_scores = np.array(all_scores)
    std_scores = np.std(all_scores, axis=0)
    avg_std = np.mean(std_scores)
    cv_scores = std_scores / np.mean(all_scores, axis=0)
    avg_cv = np.mean(cv_scores)
    print(f"\nLLM 평가의 평균 표준편차: {avg_std:.4f}")
    print(f"LLM 평가의 평균 변동계수(CV): {avg_cv:.4f}")

In [ ]:
def summarize_results(metrics_df, llm_df):
    print("\n=== 결과 요약 ===")
    
    # 모델별, 토픽 수별, 도메인별 평균 성능 비교
    for groupby_col in ['Model', 'Num_Topics', 'Domain']:
        print(f"\n{groupby_col}별 평균 성능:")
        print(metrics_df.groupby(groupby_col)[['Distinctiveness', 'JSD', 'Hellinger']].mean())
    
    # LLM 평가 결과
    if llm_df is not None:
        print("\nLLM 평가 결과:")
        print(llm_df.groupby('Model')['LLM_Avg_Score'].mean())
    
    # 최고 성능 모델
    best_models = {
        'Distinctiveness': metrics_df.loc[metrics_df['Distinctiveness'].idxmax()],
        'JSD': metrics_df.loc[metrics_df['JSD'].idxmin()],
        'Hellinger': metrics_df.loc[metrics_df['Hellinger'].idxmin()]
    }
    
    print("\n최고 성능 모델:")
    for metric, best_model in best_models.items():
        print(f"{metric}: {best_model['Model']} (토픽 수: {best_model['Num_Topics']}, 점수: {best_model[metric]:.4f})")
    
    # 결론 및 해석
    print("\n결론 및 해석:")
    print("1. 전반적으로 가장 좋은 성능을 보인 모델은 ...")
    print("2. 토픽 수에 따른 성능 변화를 보면 ...")
    print("3. 도메인별 성능 차이는 ...")
    print("4. Distinctiveness, JSD, Hellinger 지표 간의 관계는 ...")
    print("5. LLM 평가 결과와 자동 평가 지표 간의 일치도는 ...")

# 메인 실행 코드
if __name__ == '__main__':
    # 평가 지표 결과 저장을 위한 데이터프레임 초기화
    metrics_list = []

    # 계산 시간 저장을 위한 딕셔너리 초기화
    computation_times = {}

    # 모델 유형 및 토픽 수 설정
    model_types = ['LDA', 'BERTopic', 'VAE']
    num_topics_list = [2, 4]

    # 토픽 모델링 및 지표 계산
    for domain, domain_datasets in datasets.items():
        for dataset_name, data in domain_datasets.items():
            print(f"\nProcessing {domain} - {dataset_name}")
            for model_type in model_types:
                for num_topics in num_topics_list:
                    print(f"\nModel: {model_type}, Num Topics: {num_topics}")
                    try:
                        start_time = time.time()
                        if model_type == 'VAE':
                            model, vectorizer, topics = perform_topic_modeling(data, num_topics, model_type)
                        else:
                            model, vectorizer = perform_topic_modeling(data, num_topics, model_type)
                            topics = None  # VAE가 아닌 경우
                        topic_modeling_time = time.time() - start_time

                        start_time = time.time()
                        distinctiveness, jsd, hellinger = calculate_evaluation_metrics(model, data, model_type, vectorizer, num_topics, topics)
                        evaluation_time = time.time() - start_time

                        # 결과 저장
                        metrics_list.append({
                            'Domain': domain,
                            'Dataset': dataset_name,
                            'Model': model_type,
                            'Num_Topics': num_topics,
                            'Distinctiveness': distinctiveness,
                            'JSD': jsd,
                            'Hellinger': hellinger
                        })

                        computation_times[f"{domain}_{dataset_name}_{model_type}_{num_topics}"] = {
                            'Topic Modeling': topic_modeling_time,
                            'Evaluation': evaluation_time
                        }

                        print(f"Distinctiveness: {distinctiveness:.4f}")
                        print(f"JSD: {jsd:.4f}")
                        print(f"Hellinger: {hellinger:.4f}")

                    except Exception as e:
                        print(f"Error processing {domain} - {dataset_name} - {model_type} - {num_topics}: {str(e)}")
                        continue

    # metrics_df 생성
    metrics_df = pd.DataFrame(metrics_list)
    
    # 상관관계 분석 실행
    correlation_analysis(metrics_df)

    # LLM 평가 실행 (선택적)
    llm_df = llm_evaluation(metrics_df, datasets)

    # LLM 평가 결과 분석
    analyze_llm_results(llm_df)

    # LLM 평가와 자동 평가 지표 간의 상관관계 분석
    llm_auto_metric_correlation(metrics_df, llm_df)

    # LLM 평가의 일관성 검증
    verify_llm_consistency(topics, documents)

    # 결과 종합 및 해석
    summarize_results(metrics_df, llm_df)

    # 결과 분석 및 시각화 함수
    def analyze_results(metrics_df):
        metrics = ['Distinctiveness', 'JSD', 'Hellinger']
        
        # 모델별 성능 비교 및 토픽 수에 따른 성능 변화
        for metric in metrics:
            plt.figure(figsize=(12, 6))
            sns.boxplot(x='Model', y=metric, data=metrics_df)
            plt.title(f'Model {metric} Comparison')
            plt.show()

            plt.figure(figsize=(12, 6))
            sns.lineplot(x='Num_Topics', y=metric, hue='Model', data=metrics_df)
            plt.title(f'{metric} vs Number of Topics')
            plt.show()

        # 도메인별 성능 비교
        for metric in metrics:
            plt.figure(figsize=(12, 6))
            sns.boxplot(x='Domain', y=metric, data=metrics_df)
            plt.title(f'{metric} Comparison by Domain')
            plt.show()

        # 상관관계 분석
        correlation_matrix = metrics_df[metrics].corr()
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
        plt.title('Correlation between Evaluation Metrics')
        plt.show()

    # 결과 분석 실행
    analyze_results(metrics_df)

    print("실험 완료")